In [2]:
import os, sys
from src.ZillowHouseData.components.model_training import DataModeling
from src.ZillowHouseData.logger import logger
from src.ZillowHouseData.exception import CustomException
from src.ZillowHouseData.utils.common import save_model_to_keras, save_object_to_pickle, read_csv_to_dataframe, prepare_data, train_and_test_split
from src.ZillowHouseData.utils import common
from src.ZillowHouseData.config.configuration import ConfigurationManager

ModuleNotFoundError: No module named 'src'

In [ ]:
STAGE_NAME = "Data Modelling"
class DataModellingPipeline:

    def __init__(self):
        pass

    def data_model(self):
        # Create an instance of the Modelling class   
        logger.info(f">>>>>> stage {STAGE_NAME} initiated <<<<<<\n\nx==========x")
        file_path = os.path.join('artifacts', 'data_ingestion', 'final.csv')

        # Read data
        df = read_csv_to_dataframe(file_path)
        logger.info(">>>>>> CSV read <<<<<<\n\nx==========x")

        logger.info(f">>>>>> Preparing data fror training <<<<<<\n\nx==========x")
        # Preprocess data
        X, y, label_to_category_mapping = prepare_data(df)
        save_object_to_pickle(label_to_category_mapping, "models", "label")

        #Train test Split
        X_train_scaled, X_test_scaled, y_train, y_test, scaler = train_and_test_split(X, y)
        logger.info(">>>>>> Test train split completed <<<<<<\n\nx==========x")

        save_object_to_pickle( X_test_scaled, "models", "X_test_scaled.pkl")
        save_object_to_pickle( y_test, "models", "y_test.pkl")
        save_object_to_pickle(scaler, "models", "scaler.pkl")
        logger.info(">>>>>> Saved X_test and y_test as pickle for model evaluation <<<<<<\n\nx==========x")


        config = ConfigurationManager()
        model_training_config = config.get_model_training_config()
        data_modeling = DataModeling(config = model_training_config)

        # Build model
        model = data_modeling.build_model(X_train_scaled.shape[1])
        logger.info(">>>>>> Model building completed<<<<<<\n\nx==========x")

        logger.info(">>>>>> Train model <<<<<<\n\nx==========x")
        model = data_modeling.train_model(model, X_train_scaled, y_train)
        logger.info(">>>>>> Model training completed<<<<<<\n\nx==========x")

        logger.info(">>>>>> Model Summary <<<<<<\n\nx==========x\n")
        model_details = model.summary()
        logger.info(">>>>>> Model Summary <<<<<<\n\nx==========x\n\n")
        logger.info(model_details)

        save_model_to_keras(model, "models", "model.keras")
        logger.info(">>>>>> Saved model as pickle <<<<<<\n\nx==========x")


if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj2 = DataModellingPipeline()
        obj2.data_model()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise CustomException(e,sys)

In [ ]:
import os, sys, pickle
from src.ZillowHouseData.exception import CustomException
from src.ZillowHouseData.utils.common import save_object_to_pickle
from src.ZillowHouseData.entity.config_entity import ModelTrainingConfig
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
class DataModeling:
        
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        self.learning_rate = self.config.learning_rate
        self.epochs = self.config.epochs
        self.batch_size = self.config.batch_size
        self.validation_split = self.config.validation_split
        self.verbose = self.config.verbose

    def build_model(self, input_shape):
        try:
            model = Sequential()
            model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
            model.add(BatchNormalization())
            model.add(Dropout(0.5))
            model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
            model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
            model.add(Dense(1, kernel_regularizer='l2'))

            # Compile the model
            model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')

            return model

        except Exception as e:
            raise CustomException(e, sys)

    def train_model(self, model, X_train, y_train):
        try:

            params = {
                'learning_rate': self.learning_rate,
                'epochs': self.epochs,
                'batch_size': self.batch_size, 
                'validation_split': self.validation_split,
                'verbose': self.verbose
            }

            save_object_to_pickle(params, "models", "params.pkl")

            # Train the model
            model.fit(X_train, y_train, epochs=self.epochs, batch_size=self.batch_size, validation_split=self.validation_split, verbose=self.verbose)

            return model

        except Exception as e:
            raise CustomException(e, sys)



In [ ]:

def data_model_test():  
        file_path = os.path.join('artifacts', 'data_ingestion', 'final.csv')
        df = read_csv_to_dataframe(file_path)

def prepare_data_test(df):        
        X, y, label_to_category_mapping = prepare_data(df)
        save_object_to_pickle(label_to_category_mapping, "models", "label")

        X_train_scaled, X_test_scaled, y_train, y_test, scaler = train_and_test_split(X, y)

        save_object_to_pickle( X_test_scaled, "models", "X_test_scaled.pkl")
        save_object_to_pickle( y_test, "models", "y_test.pkl")
        save_object_to_pickle(scaler, "models", "scaler.pkl")

        config = ConfigurationManager()
        model_training_config = config.get_model_training_config()
        data_modeling = DataModeling(config = model_training_config)

def data_build_test():
        # Build model
        model = data_modeling.build_model(X_train_scaled.shape[1])

def train_model_test():
        model = data_modeling.train_model(model, X_train_scaled, y_train)

        model_details = model.summary()
        logger.info(model_details)

def save_model_test():
        save_model_to_keras(model, "models", "model.keras")


if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj2 = DataModellingPipeline()
        obj2.data_model()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise CustomException(e,sys)

In [7]:
import pickle
import os

# Set the path to the pickle file
pickle_file_path = "./artifacts/model/params.pkl"

# Check if the file exists
if os.path.exists(pickle_file_path):
    # Load the data from the pickle file
    with open(pickle_file_path, 'rb') as file:
        data = pickle.load(file)

    # Specify the label you want to display
    desired_label = "your_desired_label"

    # Check if the label exists in the data
    if desired_label in data:
        # Display the content of the desired label
        print(f"Content of '{desired_label}': {data[desired_label]}")
    else:
        print(f"Label '{desired_label}' not found in the pickle file.")
else:
    print(f"File '{pickle_file_path}' not found.")


File './artifacts/model/params.pkl' not found.


In [16]:
import pickle
import os

# Set the path to the pickle file
pickle_file_path = "/Users/keshavkumarelankovan/Desktop/Project_7374_/Zillow-data-ML-project/artifacts/models/label"

# Check if the file exists
if os.path.exists(pickle_file_path):
    # Load the data from the pickle file
    with open(pickle_file_path, 'rb') as file:
        data = pickle.load(file)

    # Display the content of the pickle file
    print(f"Content of '{pickle_file_path}': {data}")
else:
    print(f"File '{pickle_file_path}' not found.")


Content of '/Users/keshavkumarelankovan/Desktop/Project_7374_/Zillow-data-ML-project/artifacts/models/label': {7: 'ZATT', 9: 'ZSFH', 6: 'ZALL', 8: 'ZCON', 5: 'ZABT', 4: 'Z5BR', 1: 'Z2BR', 2: 'Z3BR', 0: 'Z1BR', 3: 'Z4BR'}
